# 交互式geomdl 

### 由geomdl计算，借助matplotlib的交互实现

In [1]:
from geomdl import BSpline
from geomdl import evaluators

# Create the curve instance
crv = BSpline.Curve()

# Set degree
crv.degree = 3

# Set control points
crv.ctrlpts = [[1, 0], [1, 1], [0, 1],[0,0],[1,-0.5],[0.5,0.5]]

# Set knot vector
crv.knotvector = [0, 0, 0,0.25,0.5,0.75,1,1,1,1]

crv.delta = 0.01
# crv.sample_size = 100

# crv.evaluator = evaluators.CurveEvaluator2()
crv.evaluate()

import numpy as np

curvePts = crv.evalpts
print(len(curvePts))

curvePts = np.array(curvePts)

import matplotlib.pyplot as plt
%matplotlib inline

crvCtlPts = np.array(crv.ctrlpts)

fig,ax = plt.subplots()
scatter1 = ax.scatter(crvCtlPts[:,0],crvCtlPts[:,1])
plot1 = ax.plot(crvCtlPts[:,0],crvCtlPts[:,1])

scatter2 = ax.scatter(curvePts[:,0],curvePts[:,1],s = 20,c=range(curvePts.shape[0]),cmap='jet')
plot2 = ax.plot(curvePts[:,0],curvePts[:,1])
fig.colorbar(scatter2)

ModuleNotFoundError: No module named 'geomdl'

In [ ]:
%matplotlib qt5 
# ok 但不是嵌入式的

import matplotlib.pyplot as plt
import numpy as np

np.set_printoptions(precision=2) #设置小数位置为3位
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

fig,ax = plt.subplots(nrows=1,ncols=1)

x = curvePts[:,0]
y = curvePts[:,1]

cx = crvCtlPts[:,0]
cy = crvCtlPts[:,1]

# 控制多边形的图元
scatter_cp = ax.scatter(cx,cy,c='r')
plot_cp, = ax.plot(cx,cy,'r')

# 曲线图元
scatter_curve = ax.scatter(x,y,c=range(curvePts.shape[0]),cmap='jet')
plot_curve, = ax.plot(x,y,'b')
fig.colorbar(scatter_curve)
    
def drawOP():    
    # 更新控制多边形图元数据
    scatter_cp.set_offsets(np.array([cx,cy]).transpose())
    plot_cp.set_data(cx,cy)
    
    # re-evaluate the curve
    crvCtlPts = np.array([cx,cy])
    crv.ctrlpts = crvCtlPts.transpose().tolist()
    crv.evaluate() # 关键：在此处更新了曲线的数据
    curvePts = np.array(crv.evalpts)
    x = curvePts[:,0]
    y = curvePts[:,1]
    
    # 更新曲线数据
    scatter_curve.set_offsets(np.array([x,y]).transpose())
    plot_curve.set_data(x,y)
    
    fig.canvas.draw_idle()
    
    
drawOP()
    
def searchpoint(x,y,xdata,ydata,epison=10):
    '''
    用像素坐标而不是数据坐标来搜索。
    '''
    global ax
    datalen = len(x)
    ptid = None
    for k in range(datalen):
        datacoord = ax.transData.transform((x[k],y[k]))
        ptcoord = ax.transData.transform((xdata,ydata))
        if np.abs(datacoord[0]- ptcoord[0]) < epison and np.abs(datacoord[1] - ptcoord[1]) < epison:
            ptid = k
            break
        
    return ptid

g_ptid = None
def onclick(event):
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))
    global g_ptid
    g_ptid = searchpoint(cx,cy,event.xdata,event.ydata)

cid = fig.canvas.mpl_connect('button_press_event', onclick)

def onrelease(event):
    print('%s release: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))
    global g_ptid
    if None != g_ptid:
        print('ptid:',g_ptid)
    else:
        print('ptid is None')
        
    g_ptid = None
    
cid = fig.canvas.mpl_connect('button_release_event',onrelease)

def updatedata(x,y,ptid,nx,ny):
    x[ptid] = nx
    y[ptid] = ny

def onmousemove(event):
    if event.button == None:
        return
    global g_ptid
    if None == g_ptid:
        return
    
    updatedata(cx,cy,g_ptid,event.xdata,event.ydata)
    
    drawOP()
    
fig.canvas.mpl_connect('motion_notify_event',onmousemove)